In [ ]:
!pip install transformers accelerate --quiet

In [ ]:
from huggingface_hub import login

login(token="")

# Google Flan:

In [10]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_id = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)

prompt = "Generate a simple 5-step animation sequence from this prompt: A spaceship landing on a mysterious alien planet. Each step will represent the context heading of a single frame of the animation."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The spaceship is landing on a mysterious alien planet.


# Mistral:

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

In [3]:
prompt = "Generate a simple 5-step animation sequence from this prompt: A spaceship landing on a mysterious alien planet. Each step will represent the context heading of a single frame of the animation."
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=300)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generate a simple 5-step animation sequence from this prompt: A spaceship landing on a mysterious alien planet. Each step will represent the context heading of a single frame of the animation.

Step 1: Approach
The spaceship soars through the star-studded cosmos, getting closer to the alien planet. Its engines emit a soft glow against the dark backdrop of space.

Step 2: Descent
The spaceship begins its descent towards the alien planet, its landing gear extending as it approaches the rocky terrain. The planet's unique atmosphere swirls around the ship, creating a mesmerizing effect.

Step 3: Touchdown
The spaceship gently touches down on the alien planet's surface, its thrusters cutting off with a soft hiss. The ship's hatch opens, revealing the eager crew members inside.

Step 4: Exploration
The crew members step out of the spaceship, their suits adjusted to the new environment. They take in their surroundings, the strange flora and fauna, and begin to collect samples for analysis.

S

# OpenChat

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "openchat/openchat-3.5-0106"  # Official release

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)

model.eval()

In [4]:
def generate_animation_frames(prompt):
    messages = [
        {"role": "system", "content": "You are a creative animation assistant who breaks story ideas into consistent and connected animation frames."},
        {"role": "user", "content": f"Convert this into 6 smooth and simple frame contexts for an animation: {prompt}"}
    ]
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)

    output = model.generate(
        input_ids=input_ids,
        max_new_tokens=300,
        do_sample=True,
        temperature=0.8,
        top_p=0.95
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [5]:
user_prompt = "A lonely robot wandering through a futuristic city at night."
result = generate_animation_frames(user_prompt)
print(result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[INST] You are a creative animation assistant who breaks story ideas into consistent and connected animation frames.

Convert this into 6 smooth and simple frame contexts for an animation: A lonely robot wandering through a futuristic city at night. [/INST] Frame 1: The sun sets over the futuristic city, casting long shadows on the streets. The lonely robot, with expressive eyes, begins to stir from its resting place.

Frame 2: The robot, equipped with a backpack, checks its map and sets off on its journey through the deserted city. Its metallic feet make soft clinking sounds as it walks.

Frame 3: The robot passes by gleaming skyscrapers, their lights twinkling in the night. It looks up at them longingly, wishing for companionship or connection.

Frame 4: The robot enters a quiet park, where it finds solace amongst the greenery. It sits down on a bench, looking at its reflection in the still pond, deep in thought.

Frame 5: Night falls completely, and the city is bathed in a soft, eth

---